In [8]:
import random

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision.transforms as T
from torchvision.datasets import CIFAR100

In [9]:
def accuracy(model, loader): #모델, 데이터로더 받아와서 정확도 계산
    model.eval()# 평가 모드로 전환
    with torch.no_grad(): # 평가시 역전파가 필요 없기 때문에 기울기 계산하지 않음
        total, correct = 0, 0 #전체 데이터 개수, 맞춘 개수
        for imgs, targets in loader: #입력 이미지, 정답 라벨을 데이터 로더에서 받아옴.
            imgs = imgs.cuda()
            targets = targets.cuda()

            logits = model(imgs) #모델에 데이터 넣고 출력값 계산
            _, preds = torch.max(logits.data, 1) #logits에서 각 샘플당 하나씩 가장 큰 값 뽑기아 예측값에 넣기.

            total += targets.size(0)
            #모든 배치의 샘플 수를 누적해서 전체 평가 개수 세기.
            #배치 사이즈로 데이터 나뉘니깐.
            correct += (preds == targets).sum().item()
            # 예측값이랑 정답 비교해서 맞춘 개수를 누적.

        return 100 * correct / total # 맞춘개수/전체데이터*100

In [10]:
EPOCH = 1500
BATCH_SIZE = 128
# 너무 작으면 학습이 느려지고 변동성이 크다.
# 너무 크면 메모리 부족 or 일반화 성능이 낮아질 수 있다.

In [11]:
train_transform = T.Compose([
            T.RandomCrop(32,padding=4),
            # 이미지를 4픽셀씩 패딩한 후, 32x32로 랜덤하게 자른다.
            # padding = 4 를 한 이유
            # 1.데이터가 작기 때문에 overfitting 방지에 효과적이다.
            # 2.CIFAR-10/100 논문에서도 4가 표준처럼 쓰임.
            T.RandomHorizontalFlip(),# 50% 확률로 좌우반전
            # 사용 이유
            # 1.overfitting 방지
            # 1_1.overfitting : 학습 데이터에만 너무 특화되어서 새로운 데이터에 잘 일반화 하지 못하는 현상
            # 1_2.하나의 이미지가 두개의 학슴 샘플이 된다. -> 데이터가 더 풍부해지고 모델이 다양한 상황을 학습 -> 과적합 가능성 줄어듦.
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),# (밝기,대비,채도,색조)
            # 사용 이유
            # 1.다양한 조명/화질 상황에서도 잘 작동하도록 도와줌.
            # 2.색상에 대한 overfitting 방지
            # 3.너무 많은 변화는 다양성은 올라가지만, 학습 성능에 해가 될 수 있음.
            # 3_1.brightness(조명 변화 정도):0.1~0.3 추천
            # 3_2.contrast(흐릿함~뚜렷함 정도):0.1~0.3 추천
            # 3_3.saturation(색의 진하기 변화):0.1~0.3 추천
            # 3_4.hue(살짝 다른 색으로 변환/과하면 색이 이상해짐): <= 0.1 추천
            T.RandomAffine(degrees=15, translate=(0.1, 0.1), scale =(0.95,1.05),shear=5),
            # 사용 이유
            # 1.overfitting 방지
            # 2.객체가 가질 수 있는 다양한 변형에 대해 일반화된 이해를 갖도록 유도
            # 3.RandomAffine : 이미지를 무작위로 회전,이동,확대/축소, 기울기 변환 적용.
            # 3_1.degrees(회전) : 10 or 15 가 안전하고 효과적.
            # 3_2.translate(이동) : (좌우 이동 비율, 상하 이동 비율), (0.1,0.1)추천
            # 3_2_1.너무 많이 translate하면 이미지에서 객체가 사라지는 문제 생김.
            # 3_3.scale(확대/축소) : (확대,축소),(0.95,1.05) 추천
            # 3_4.shear(기울기) : 5 추천.
            T.ToTensor(),
            # 이미지를 Pytorch가 처리할 수 있는 torch.Tensor 타입으로 변환
            # 데이터 타입 변환, 픽셀 값 정규화, 차원 순서 변경 등 이미지를 신경망에 입력하기 위해 필요한 변환.
            T.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
            # 이미지 데이터 정규화
            # (rgb 평균, rgb 표준편차) / 위 값은 CIFAR-100 전체 학습 이미지 50,000장의 각 채널별 평균과 표준편차(논문에서 사용)
            # 정규화 사용 이유
            # 1.학습 안정성 향상
            # 1_1.입력 값이 정규화되어야 Gradient Descent 최적화가 더 잘 작동
            # 1_2.정규화 하지 않으면 편향 생길수 있음.
            # 2.빠른 수렴
            # 2_1.데이터가 비슷한 범위에 있으면 파라미터 업데이트가 안정적이고 빠르게 이뤄짐.
            # 2_2.학습 속도 빨리지고, 수렴도 잘 됨.
            # 3.일반화 성능 향상
            # 3_1.정규화를 통해 모델이 입력 분포에 민감하지 않도록 만들어줌.
        ])
## transform 하는 이유
# 1.훈견 데이터를 다양하게 변형해서 일반화 성능을 높이고, overfitting 방지
test_transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
    # test에도 Normalize 해줘야함. test와 분포가 같아야해서.

])

train = CIFAR100('./', train=True, download=True, transform=train_transform)
test = CIFAR100('./', train=False, download=False, transform=test_transform)
## train과 test의 transform이 다른 이유
# 1.train은 모델을 학습시키는 것/ test는 모델이 잘 작동하는지 보는 것.
# 2.train은 다양하게 agumentation을 하고, test에는 절대로 변형을 하면 안된다.변환만 해줌.
# 2_2.이유는 원본을 유지해야 정확한 일반화 성능을 측정 가능하므로.
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)
test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)
# 1.shuffle : 데이터를 epoch마다 섞어주는 옵션
# 1_1.모델이 데이터 순서에 의존하지 않게 해줌.
# 1_2.학습 안정성+일반화 성능 향상
# 2.num_workers : 데이터 로딩을 병렬로 처리할 스레드 수
# 2_1.속도 향상
# 2_2.0일 경우 데이터를 하나씩 순차적으로 부름.
# 2_3.n개 작업을 동시에 불러옴(빠름)/추천:2~8

Files already downloaded and verified


In [12]:
model = nn.Sequential(# CNN model 완성

    nn.Conv2d(3,32,3,1,1),
    # 1.Conv2d(in_channel,out_channel,kernel_size,stride,padding)
    # 2.이미지의 공간적 특징을 살리면서, 그안에서 특징을 추출하기 위해 사용(선형구조)
    # 사용이유
    # 1.단순한 특징 -> 복잡한 특징으로 발전
    # 2.표현력을 높이기 위해
    # 3.정보 손실 보완 + 추상화 강화
    nn.BatchNorm2d(32),
    # 사용이유
    # 1.학습 속도 향상:값의 분포가 안정돼서 학습이 빨라짐
    # 2.기울기 소실,발산 문제 완화
    # 3.overfitting 방지
    # 4.Conv 연산 후 feature map의 값 분포가 층마다 바뀌면서 불안정해지는데 잡아줌
    nn.LeakyReLU(inplace=False),
    # 사용이유
    # 1.입력 데이터를 잠재 벡터로 압축. 비선형성을 부가해 더 복잡한 패턴 학습할 수 있다.
    # 2.relu는 음수를 다 0으로 만들기 때문에, 뉴런이 완전히 죽을수 있다.
    # 3.leakrelu는 음수일때도 작은 기울기로 정보가 조금씩 흐를수있게 , 안정적으로 학습하게 도와준다.
    # 4.inplace=False 이건 원본을 유지하고 새로 계산된 값 반환. True면 기존값 덮어씀.
    # 5.True하면 원본값 날라가서 역전파에서 그 값을 기반으로 오차의 gradient 계산 못함.

    nn.Conv2d(32,64,3,1,1),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(inplace=False),
    nn.Dropout(0.3),
    # 사용이유
    # 1.overfitting 방지
    # 2.CIFAR-100과 같은 작은 이미지 데이터셋은 쉽게 과적합 되므로 추천.
    # 3.일반화 성능 향상

    nn.Conv2d(64,128,3,2,1),
    # 요즘 pooling 안써서 stride 2 로 해줌
    # 연산과 downsampling을 동시에 한다.
    # stride=2 하여 공간을 줄이는 이유
    # 1.모델을 계층적으로 추상화
    # 2.계산량 감소 -> 채널은 늘고, 크기는 줄면서 효율적인 구조가 된다.
    # 3.과적합 방지 -> 공간을 줄이면 불필요한 정보 버리고 중요한 것 만 추출
    # 4.네트워크가 깊어진다면 공간을 줄이는게 필수적임.(3층 이상)
    nn.BatchNorm2d(128),
    nn.LeakyReLU(inplace=False),
    nn.Dropout(0.3),

    nn.Conv2d(128,256,3,2,1),
    nn.BatchNorm2d(256),
    # 1.마지막층에 LeakyRelu 사용하지 않는 이유
    # 1_1.비선형성은 단선한 선형 특징을 조합해서 더 복잡하고 유의미한 특징을 추출하게 해준다.
    # 1_2.특징 추출이 끝나서 비선형성을 추가할 필요가 없음
    # 2.마지막층에 Dropout 사용하지 않는 이유
    # 2_1.마지막에 추가하면 정보가 지나치게 소실됨 -> 분류 성능 저하

    nn.AdaptiveAvgPool2d(1),
    # 사용이유
    # 1.각 채널별로 평균값 하나만 남김.
    # 2.위치 정보가 필요 없어서(분류라서 위치 정보 안중요함.) 존재 강도(존재 여부)만 요약.
    # 3.연산량이 확 줄고, 오버피팅이 준다. 결과는 항상 (c,1,1)로 뽑혀서 연산량이 작아짐. -> 모델이 작아짐.
    # 4.파라미터가 줄어서 overfitting 방지, 학습이 잘되고, 결과도 잘나옴.
    # 5.어떤 이미지 크기든 출력 벡터 크기가 같아 입력 크기에 유연하다.
    nn.Flatten(),
    # feature map 1차원으로 펼치기.
    # 사용 이유
    # 1.Linear는 1차원 벡터 입력만 받는다.(선형)
    # 2.공간 정보를 없애고, 예측을 위한 벡터로 만든다.
    nn.Linear(256,100),
    # CIFAR-100 데이터셋은 클래스 수가 100개라서.
)
model = model.cuda()

In [13]:
loss_function = torch.nn.CrossEntropyLoss().cuda()
# CrossEntropyLoss 사용 이유
# 1.다중 클래스 분류 문제에서 가장 일반적으로 사용하는 손실함수
optimizer = torch.optim.Adam(model.parameters())
# 가중치 조정

In [14]:
for epoch in range(EPOCH):
    model.train()# 생성자 학습모드로 설정
    for img, target in train_loader:#이미지와 라벨정보 들고옴
        img = img.cuda()
        target = target.cuda()
        logit = model(img)#이미지를 model에 적용하여 예측값 생성

        optimizer.zero_grad()
        # 기울기 초기화 : 하지 않으면 이상한 방향으로 학습됨.

        loss = loss_function(logit, target)#예측값 정답 비교해 손실 계산

        loss.backward()
        # 역전파 : 각 가중치가 손실에 얼마나 영향을 주었는지 계산해서 다음번 가중치 업데이트 방향 결정.
        # 손실값을 가지고 어떻게 가중치를 업데이트할지 결정한다.
        optimizer.step()
        # 역전파가 알려준 만큼 가중치 업데이트


    if epoch % 50 == 0: #epoch50번 진행마다 학습/테스트 정확도 표기
        print(f'{epoch + 1} epoch: {accuracy(model, train_loader)} / {accuracy(model, test_loader)}')

1 epoch: 11.494 / 12.75
51 epoch: 45.324 / 45.09
101 epoch: 48.996 / 48.91
151 epoch: 51.018 / 51.09
201 epoch: 51.872 / 51.2
251 epoch: 52.616 / 52.44
301 epoch: 53.41 / 53.05
351 epoch: 53.874 / 52.56
401 epoch: 53.578 / 53.61
451 epoch: 54.032 / 53.51
501 epoch: 54.546 / 53.55
551 epoch: 54.944 / 54.19
601 epoch: 54.602 / 53.27
651 epoch: 54.716 / 54.17
701 epoch: 55.256 / 54.32
751 epoch: 54.942 / 54.39
801 epoch: 55.058 / 54.25
851 epoch: 55.564 / 54.63
901 epoch: 55.796 / 54.88
951 epoch: 55.646 / 54.62
1001 epoch: 56.136 / 54.93
1051 epoch: 55.994 / 55.02
1101 epoch: 55.924 / 55.35
1151 epoch: 55.978 / 54.73
1201 epoch: 56.406 / 55.3
1251 epoch: 56.65 / 55.09
1301 epoch: 56.406 / 55.29
1351 epoch: 56.56 / 55.14
1401 epoch: 56.88 / 55.43
1451 epoch: 56.462 / 55.21
